In [1]:
import random
from functools import reduce
from __future__ import division
from scipy import stats 
from scipy.stats import multivariate_normal as m_n
import numpy as np
from  math import log

In [2]:
def dot(v,u):
    return sum(i*j for (i,j) in zip(v,u))

In [3]:
def add(v,u):
    return [i+j for (i,j) in zip(v,u)]

In [4]:
def nu(a, u):
    return [a*i for i in u ]

In [5]:
def nu_vec(u ,v):
    return [i*j  for (i,j) in zip(u, v)]

In [6]:
def vec_fi(v):
    return[[i] for i in v]

In [7]:
def vec_sum(v):
    return reduce(add, v)

In [8]:
def vec_mean(v):
    return un(1/max(1, len(v)), vec_sum(v))

In [9]:
def dist(v,u):
    return dot(add(v, nu(-1,u)), add(v, nu(-1,u)))**0.5

In [10]:
def mat_dot(m,n):
    l =  inver(n)
    return[[dot(i, j) for j in l] for i in m ]

In [11]:
def make_mat(u, v):
    return mat_dot(vec_fi(u), inver(vec_fi(v)))

In [12]:
def inver(d):
    return [ [d[j][i] for j in range(len(d)) ] for i in range(len(d[0]))]

In [13]:
def trace(m):
    dig = [m[i][i] for i in range(len(m))]
    return sum(dig)

In [14]:
def ge_pi (k):
    l = [random.randint(0, 10000) for i in range(k-1)]
    l_p = sorted(l)+[10000]
    l_m =[0]+sorted(l)
    l_re = [i-j for (i,j) in zip(l_p , l_m)]
    return [i/10000 for i in l_re]

In [15]:
def ge_pi2 (k):
    l = [random.randint(0, 10000/(k-1)) for i in range(k-1)]
    l_re=[ i-j+(10000/(k-1))  for (i,j) in zip(l+[0], [10000/(k-1)]+l)]
    return [ i/10000 for i in l_re]

In [16]:
def ge_inv_mat (k):
    mat=[[0 for i in range(k)] for j in range (k)]
    for i in range (k):
        for j in range(k):
            if i==j:
                mat[i][j]=0
            else:
                mat[i][j]=random.uniform(100, 101)
    return mat

In [59]:
def ge_posi_mat (k):
    p = np.array(ge_inv_mat(k))
    p1=np.linalg.inv(p)
    cov = [[0 for j in range(k) ] for i in range(k)]
    for i in range(k):
        for j in range(k):
            if i==j:
                cov[i][j]=random.uniform(5, 10)
            else:
                cov[i][j]=0
    return reduce(mat_dot, [p1, cov, p])

In [18]:
def mat_sum(u):
    re_mat = [[j[i] for j in u] for i in  range(len(u[0]))]
    return [vec_sum(i) for i in re_mat]

In [19]:
def make_mat_from(data):
    re_data = [make_mat(i, i)  for i in data]
    return mat_sum(re_data)

In [20]:
def nu_mat(a ,mat):
    return[nu(a, i) for i in mat]

In [21]:
def ge_cov(k):
    vec = ge_inv_mat(k)
    mat_ind= make_mat_from(vec)
    return nu_mat(1/1, mat_ind) 

In [22]:
def comp_gam(data, c, cov, pi):
    gam_data=[[m_n.pdf(i, j, k)*l for (j,k,l) in zip(c, cov, pi)] for i in data]
    return inver([[(j+0.00001)/(sum(i)+0.00001*(len(pi)+1)) for j in i] for i in gam_data])

In [23]:
def c_upd(data, gam, n_k):
    kdata = [[ nu_vec(j, i)  for i in data] for j in gam]
    return [nu(1/j, vec_sum(i)) for (i,j) in zip(kdata, n_k)]

In [24]:
def cov_upd(data, c, gma, n_k):
    data1 = [[add(j ,nu(-1, i)) for j in data] for i in c]
    data2=[[nu(k,l) for (k,l) in zip(i,j)] for (i,j) in zip(gma, data1)]
    mat_unsum=[[make_mat(i, j) for (i,j) in zip(k,l)] for (k,l) in zip(data1, data2)]
    mat =[mat_sum(i) for i in mat_unsum]
    return [nu_mat(1/i, j) for (i,j) in zip(n_k, mat)]

In [25]:
def likhod(data, c, cov_l, pi):
    data_likhod = [ sum([m_n.pdf(i, j, k)*l for (j,k,l) in zip (c, cov_l, pi) ]) for i in data]
    return reduce( lambda x, y: x*y, data_likhod)

In [60]:
def m_g (data , k):
    dem = len(data[0])
    para_list=[]
    for i in range(len(data)//100+5):
        c =random.sample(data, k)
        pi = ge_pi(k)
        cov_l = [ge_cov(dem) for i in range(k)]
        while True:
            gam = comp_gam(data, c, cov_l, pi)
            n_k = [sum(i) for i in gam]
            cnew = c_upd(data, gam, n_k)
            cov_l = cov_upd(data, c, gam, n_k)
            pi = [i/sum(n_k) for i in n_k]
            if max([dist(i,j)/(max(dot(i,i)**0.5, 0.00001))  for (i,j) in zip(c, cnew) ]) < 000000.1:
                break
            c = cnew
        para_list.append([c, cov_l, pi])
    ll = [likhod(data, i[0], i[1], i[2]) for i in para_list]
    ind = ll.index(max(ll))
    return para_list[ind]

In [61]:
m_g(data, 3)

[[[1216.5442868371476, 2846.0614188914428],
  [1216.544234304512, 2846.06129599324],
  [1216.5444385880865, 2846.0617739072813]],
 [[[67000455.702895984, -1990898.0515764605],
   [-1990898.0515764605, 333300669.89411312]],
  [[67000453.640862219, -1990897.9349857173],
   [-1990897.9349857173, 333300659.59005362]],
  [[67000459.007798217, -1990898.2383971242],
   [-1990898.2383971242, 333300686.40906203]]],
 [0.33333333545453009, 0.33333334601264292, 0.33333331853282699]]

In [16]:
print multivariate_normal.__doc__ 


    A multivariate normal random variable.

    The `mean` keyword specifies the mean. The `cov` keyword specifies the
    covariance matrix.

    Methods
    -------
    ``pdf(x, mean=None, cov=1, allow_singular=False)``
        Probability density function.
    ``logpdf(x, mean=None, cov=1, allow_singular=False)``
        Log of the probability density function.
    ``rvs(mean=None, cov=1, size=1, random_state=None)``
        Draw random samples from a multivariate normal distribution.
    ``entropy()``
        Compute the differential entropy of the multivariate normal.

    Parameters
    ----------
    x : array_like
        Quantiles, with the last axis of `x` denoting the components.
    mean : array_like, optional
        Mean of the distribution (default zero)
    cov : array_like, optional
        Covariance matrix of the distribution (default one)
    allow_singular : bool, optional
        Whether to allow a singular covariance matrix.  (Default: False)
    random_state : 

In [163]:
max([1,2])

2

In [169]:
len(comp_gam(data, c, cov, pi)[0])

660

In [111]:
nk =[sum(i) for i in ga]

In [50]:
pi=ge_pi(5)

In [51]:
c = random.sample(data, 5)

In [154]:
ga =comp_gam(data, c , [ ge_cov(2) for i in range(5)], pi)

In [155]:
c_upd(data, ga, nk)

[[1220.0645028248134, 2854.2968370484823],
 [1139.6784982096708, 2666.2366827003038],
 [1479.9719336759817, 3462.34088397082],
 [1128.399301824335, 2639.8494101482561],
 [1142.372958745904, 2672.5402758040386]]

In [ ]:
data1 = [[add(j ,nu(-1, i)) for j in data] for i in c]
mat_unsum=[[make_mat(i, j) for (i,j) in zip(k,l)] for (k,l) in zip(data, data1)]
mat =[mat_sum(i) for i in mat_unsum]

In [133]:
data2=[[nu(k,l) for (k,l) in zip(i,j)] for (i,j) in zip(ga, data1)]

In [137]:
mat_unsum=[[make_mat(i, j) for (i,j) in zip(k,l)] for (k,l) in zip(data1, data2)]
mat =[mat_sum(i) for i in mat_unsum]

In [138]:
mat

[[[7532909765.1766682, 144857729.01344502],
  [144857729.01344502, 37475652641.916969]],
 [[7532908513.5058498, 116759739.22086191],
  [116759739.22086191, 37354963556.668289]],
 [[7532908591.871418, 161719340.75979838],
  [161719340.75979838, 37552589107.637901]],
 [[7532906092.4780254, -25465727.735943757],
  [-25465727.735943757, 36888733935.370552]],
 [[7532905562.9925022, -636653220.64779615],
  [-636653220.64779615, 37638646206.336975]]]

In [141]:
[nu(1/i, j) for (i,j) in zip(nk, mat)]

C:\Anaconda2\lib\site-packages\ipykernel\__main__.py:2: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


[[[], []], [[], []], [[], []], [[], []], [[], []]]

In [142]:
nk

[112.29896904292904,
 105.8505774037947,
 124.17713496246684,
 106.78628624445534,
 105.48006228625992]

In [156]:
cov_l =cov_upd(data, c, ga, nk)

In [157]:
likhod(data, c, cov_l, ge_pi(5))

ValueError: math domain error

In [64]:
[add([0.1, 0.1], i)  for i in random.sample(data, 2)]

[[0.1, 490.1], [0.1, 1226.1]]

In [66]:
[ ge_cov(2) for i in range(2)]

[[[0.4818189468827886, 0.0], [0.0, 0.3425863968697871]],
 [[0.37026537147715705, 0.0], [0.0, 0.08384262866215438]]]

In [69]:
ge_pi(2)

[0.2966, 0.7034]

In [92]:
def comp_gam(data, c, cov_list, pi):
    return [[m_n.pdf(i, j, k)*l for (j,k,l) in zip(c, cov_list, pi)] for i in data]

In [ ]:
comp_gam( )

In [79]:
a =[[0.1, 490.1], [0.1, 1226.1]]
b= [[[0.4818189468827886, 0.0], [0.0, 0.3425863968697871]],
 [[0.37026537147715705, 0.0], [0.0, 0.08384262866215438]]]

In [82]:
b

[[[0.4818189468827886, 0.0], [0.0, 0.3425863968697871]],
 [[0.37026537147715705, 0.0], [0.0, 0.08384262866215438]]]

In [83]:
[(j,k) for (j, k) in (a, b)] 

[([0.1, 490.1], [0.1, 1226.1]),
 ([[0.4818189468827886, 0.0], [0.0, 0.3425863968697871]],
  [[0.37026537147715705, 0.0], [0.0, 0.08384262866215438]])]

In [ ]:
[(i,j) for (i,j) in ()]

In [55]:
p = np.array(ge_cov(3))
p1=np.linalg.inv(p)

## [ge_cov(2) for i in range(10)]

In [28]:
data=[[0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 0],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 1],
 [0, 2],
 [0, 2],
 [0, 2],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 3],
 [0, 4],
 [0, 4],
 [0, 4],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 5],
 [0, 6],
 [0, 6],
 [0, 6],
 [0, 7],
 [0, 7],
 [0, 8],
 [0, 8],
 [0, 11],
 [0, 12],
 [0, 12],
 [0, 12],
 [0, 13],
 [0, 13],
 [0, 13],
 [0, 14],
 [0, 14],
 [0, 14],
 [0, 14],
 [0, 15],
 [0, 16],
 [0, 16],
 [0, 19],
 [0, 19],
 [0, 19],
 [0, 20],
 [0, 21],
 [0, 24],
 [0, 26],
 [0, 27],
 [0, 27],
 [0, 28],
 [0, 28],
 [0, 30],
 [0, 30],
 [0, 32],
 [0, 35],
 [0, 35],
 [0, 38],
 [0, 39],
 [0, 39],
 [0, 39],
 [0, 41],
 [0, 42],
 [0, 42],
 [0, 43],
 [0, 43],
 [0, 44],
 [0, 45],
 [0, 45],
 [0, 46],
 [0, 48],
 [0, 48],
 [0, 48],
 [0, 49],
 [0, 50],
 [0, 50],
 [0, 51],
 [0, 52],
 [0, 54],
 [0, 55],
 [0, 58],
 [0, 59],
 [0, 59],
 [0, 60],
 [0, 61],
 [0, 61],
 [0, 62],
 [0, 63],
 [0, 64],
 [0, 66],
 [0, 66],
 [0, 67],
 [0, 70],
 [0, 73],
 [0, 75],
 [0, 75],
 [0, 77],
 [0, 78],
 [0, 80],
 [0, 84],
 [0, 88],
 [0, 91],
 [0, 94],
 [0, 99],
 [0, 102],
 [0, 104],
 [0, 104],
 [0, 105],
 [0, 108],
 [0, 108],
 [0, 109],
 [0, 109],
 [0, 114],
 [0, 114],
 [0, 115],
 [0, 120],
 [0, 121],
 [0, 121],
 [0, 122],
 [0, 123],
 [0, 127],
 [0, 128],
 [0, 129],
 [0, 129],
 [0, 130],
 [0, 130],
 [0, 134],
 [0, 135],
 [0, 135],
 [0, 138],
 [0, 143],
 [0, 149],
 [0, 154],
 [0, 154],
 [0, 155],
 [0, 156],
 [0, 158],
 [0, 160],
 [0, 163],
 [0, 165],
 [0, 166],
 [0, 166],
 [0, 167],
 [0, 171],
 [0, 180],
 [0, 180],
 [0, 182],
 [0, 184],
 [0, 185],
 [0, 196],
 [0, 202],
 [0, 204],
 [0, 207],
 [0, 212],
 [0, 226],
 [0, 235],
 [0, 238],
 [0, 251],
 [0, 252],
 [0, 255],
 [0, 257],
 [0, 263],
 [0, 265],
 [0, 266],
 [0, 270],
 [0, 272],
 [0, 280],
 [0, 292],
 [0, 301],
 [0, 301],
 [0, 308],
 [0, 317],
 [0, 321],
 [0, 326],
 [0, 326],
 [0, 327],
 [0, 333],
 [0, 337],
 [0, 338],
 [0, 341],
 [0, 345],
 [0, 345],
 [0, 346],
 [0, 351],
 [0, 353],
 [0, 360],
 [0, 363],
 [0, 363],
 [0, 367],
 [0, 370],
 [0, 373],
 [0, 377],
 [0, 377],
 [0, 377],
 [0, 379],
 [0, 400],
 [0, 401],
 [0, 403],
 [0, 404],
 [0, 415],
 [0, 415],
 [0, 416],
 [0, 421],
 [0, 423],
 [0, 427],
 [0, 433],
 [0, 439],
 [0, 444],
 [0, 461],
 [0, 469],
 [0, 475],
 [0, 478],
 [0, 478],
 [0, 480],
 [0, 481],
 [0, 490],
 [0, 495],
 [0, 504],
 [0, 517],
 [0, 521],
 [0, 524],
 [0, 545],
 [0, 550],
 [0, 552],
 [0, 563],
 [0, 565],
 [0, 581],
 [0, 582],
 [0, 590],
 [0, 596],
 [0, 603],
 [0, 611],
 [0, 614],
 [0, 620],
 [0, 621],
 [0, 624],
 [0, 634],
 [0, 652],
 [0, 666],
 [0, 667],
 [0, 668],
 [0, 671],
 [0, 674],
 [0, 675],
 [0, 679],
 [0, 680],
 [0, 687],
 [0, 695],
 [0, 699],
 [0, 700],
 [0, 704],
 [0, 727],
 [0, 738],
 [0, 743],
 [0, 750],
 [0, 757],
 [0, 774],
 [0, 789],
 [0, 797],
 [0, 803],
 [0, 814],
 [0, 817],
 [0, 819],
 [0, 830],
 [0, 833],
 [0, 891],
 [0, 896],
 [0, 916],
 [0, 947],
 [0, 947],
 [0, 986],
 [0, 988],
 [0, 989],
 [0, 994],
 [0, 1022],
 [0, 1030],
 [0, 1052],
 [0, 1054],
 [0, 1067],
 [0, 1078],
 [0, 1092],
 [0, 1119],
 [0, 1122],
 [0, 1147],
 [0, 1190],
 [0, 1200],
 [0, 1226],
 [0, 1247],
 [0, 1283],
 [0, 1318],
 [0, 1358],
 [0, 1358],
 [0, 1369],
 [0, 1400],
 [0, 1437],
 [0, 1454],
 [0, 1468],
 [0, 1498],
 [0, 1505],
 [0, 1535],
 [0, 1545],
 [0, 1580],
 [0, 1628],
 [0, 1667],
 [0, 1679],
 [0, 1695],
 [0, 1701],
 [0, 1710],
 [0, 1730],
 [0, 1798],
 [0, 1839],
 [0, 1861],
 [0, 1927],
 [0, 1955],
 [0, 2007],
 [0, 2049],
 [0, 2051],
 [0, 2090],
 [0, 2161],
 [0, 2233],
 [0, 2334],
 [0, 2477],
 [0, 2485],
 [0, 2683],
 [0, 2820],
 [0, 3023],
 [0, 3076],
 [0, 3205],
 [0, 3295],
 [0, 3338],
 [0, 3341],
 [0, 3423],
 [0, 3849],
 [0, 3868],
 [0, 3868],
 [0, 3955],
 [0, 4296],
 [0, 4393],
 [0, 4433],
 [0, 4602],
 [0, 4719],
 [0, 4855],
 [0, 5016],
 [0, 5169],
 [0, 5350],
 [0, 5968],
 [0, 6042],
 [0, 6962],
 [0, 7925],
 [0, 8010],
 [0, 8049],
 [0, 8149],
 [0, 9180],
 [0, 9421],
 [0, 9726],
 [0, 11237],
 [0, 12232],
 [0, 12507],
 [0, 15563],
 [0, 22606],
 [0, 96484],
 [1, 0],
 [1, 2],
 [1, 15],
 [1, 26],
 [1, 29],
 [1, 75],
 [1, 163],
 [1, 193],
 [1, 195],
 [1, 222],
 [1, 239],
 [1, 243],
 [1, 268],
 [1, 371],
 [1, 402],
 [1, 403],
 [1, 407],
 [1, 409],
 [1, 412],
 [1, 444],
 [1, 504],
 [1, 506],
 [1, 571],
 [1, 576],
 [1, 608],
 [1, 685],
 [1, 770],
 [1, 814],
 [1, 933],
 [1, 1094],
 [1, 1631],
 [1, 2523],
 [1, 2640],
 [1, 3240],
 [1, 3894],
 [1, 3941],
 [1, 5813],
 [1, 7967],
 [1, 8951],
 [1, 9241],
 [1, 14970],
 [1, 29981],
 [1, 52209],
 [2, 34],
 [2, 52],
 [2, 93],
 [2, 205],
 [2, 246],
 [2, 338],
 [2, 387],
 [2, 570],
 [2, 1152],
 [2, 2579],
 [2, 129356],
 [2, 379672],
 [3, 10],
 [3, 22],
 [3, 37],
 [3, 101],
 [3, 108],
 [3, 151],
 [3, 160],
 [3, 215],
 [3, 243],
 [3, 725],
 [3, 825],
 [3, 2127],
 [3, 4452],
 [3, 93060],
 [4, 7],
 [4, 12],
 [4, 84],
 [4, 290],
 [4, 324],
 [4, 358],
 [4, 475],
 [4, 580],
 [5, 406],
 [5, 596],
 [5, 731],
 [5, 1000],
 [5, 4309],
 [6, 146],
 [6, 1094],
 [6, 2577],
 [7, 1],
 [7, 64],
 [7, 243],
 [7, 4135],
 [7, 4603],
 [8, 28],
 [8, 375],
 [8, 524],
 [8, 1143],
 [8, 1735],
 [9, 915],
 [10, 106],
 [10, 243],
 [10, 11845],
 [11, 128],
 [11, 199],
 [11, 1290],
 [12, 0],
 [12, 38],
 [12, 92],
 [12, 99],
 [12, 357],
 [12, 1046],
 [13, 119],
 [13, 2731],
 [14, 57],
 [14, 474],
 [14, 848],
 [14, 1261],
 [15, 48],
 [15, 101],
 [15, 204],
 [15, 1126],
 [16, 654],
 [17, 2748],
 [17, 13298],
 [18, 0],
 [18, 27410],
 [19, 148],
 [21, 1062],
 [23, 1143],
 [24, 89],
 [24, 137],
 [24, 1255],
 [25, 405],
 [26, 25594],
 [28, 3],
 [28, 13366],
 [29, 115],
 [29, 1405],
 [30, 166],
 [30, 436],
 [31, 4],
 [31, 354],
 [31, 857],
 [31, 1254],
 [31, 1431],
 [35, 391],
 [36, 14],
 [36, 599],
 [37, 64],
 [40, 16],
 [40, 1398],
 [41, 1896],
 [43, 245],
 [44, 27],
 [46, 77],
 [46, 863],
 [48, 764],
 [50, 11],
 [50, 201],
 [50, 1307],
 [52, 648],
 [52, 981],
 [54, 675],
 [55, 943],
 [58, 1506],
 [67, 2335],
 [70, 1655],
 [70, 9369],
 [71, 248],
 [73, 738],
 [73, 1063],
 [77, 66],
 [78, 64],
 [79, 59],
 [79, 2972],
 [80, 44],
 [80, 9258],
 [84, 24],
 [89, 50],
 [95, 262],
 [103, 142],
 [104, 337],
 [105, 49],
 [108, 159],
 [109, 338],
 [110, 1117],
 [110, 1858],
 [114, 128],
 [116, 141217],
 [121, 705],
 [126, 797],
 [128, 2633],
 [130, 17],
 [132, 1],
 [153, 1],
 [155, 2084],
 [173, 29017],
 [176, 149],
 [177, 868],
 [179, 106],
 [180, 10425],
 [188, 2659],
 [190, 103],
 [191, 278],
 [192, 4031],
 [194, 300],
 [212, 33],
 [212, 178],
 [213, 62],
 [227, 89],
 [240, 125405],
 [244, 2266],
 [247, 47],
 [252, 123],
 [258, 394],
 [297, 9904],
 [305, 14],
 [308, 0],
 [315, 211],
 [318, 239],
 [361, 242],
 [368, 526],
 [383, 377],
 [407, 87],
 [438, 0],
 [447, 0],
 [456, 202],
 [457, 250],
 [566, 6],
 [587, 36],
 [612, 3360],
 [618, 1198],
 [650, 50],
 [667, 276],
 [693, 446],
 [734, 16876],
 [760, 2257],
 [761, 156],
 [822, 223],
 [826, 124],
 [847, 1887],
 [855, 1411],
 [864, 177],
 [920, 115],
 [935, 849],
 [1006, 175],
 [1015, 51],
 [1051, 303],
 [1118, 45],
 [1158, 11972],
 [1235, 1],
 [1354, 82],
 [1428, 55],
 [1457, 2629],
 [1462, 117],
 [1504, 23],
 [1760, 1363],
 [1825, 1341],
 [2207, 103],
 [2803, 0],
 [2904, 496],
 [2990, 182],
 [3247, 81],
 [3571, 121],
 [3737, 16361],
 [3858, 62],
 [4059, 1332],
 [4431, 11],
 [4818, 1355],
 [5342, 433],
 [5583, 234],
 [6541, 508],
 [6614, 522],
 [6709, 78],
 [7273, 202],
 [8402, 13077],
 [8405, 492],
 [8501, 35],
 [8566, 154],
 [8681, 111],
 [8794, 1],
 [9130, 646],
 [11888, 395],
 [12720, 720],
 [30634, 64],
 [35960, 2],
 [36029, 306],
 [36280, 104],
 [36964, 292],
 [59762, 1],
 [63948, 644],
 [81014, 718],
 [102176, 33],
 [114335, 4399]]